In [1]:
include("../src/Categories.jl")
include("../src/FinSets.jl")

Main.FinSets

In [2]:
using .Categories, .FinSets

In [3]:
struct Graph
    vertices::FinSet{Symbol}
    edges::FinSet{Symbol}
    src::Dict{Symbol,Symbol}
    tgt::Dict{Symbol,Symbol}
end

In [4]:
struct ACSet
    schema::Graph
    nextId::Ref{Int}
    parts::Dict{Symbol, FinSet{Int}}
    subparts::Dict{Symbol, Dict{Int,Int}}
    function ACSet(schema::Graph)
        new(
            schema,
            Ref(0),
            Dict{Symbol,FinSet{Int}}(ob => Set{Int}() for ob in schema.vertices),
            Dict{Symbol,Dict{Int,Int}}(hom => Dict{Int,Int}() for hom in schema.edges)
        )
    end
end

In [5]:
Gr = Graph(Set([:V,:E]), Set([:src,:tgt]), Dict(:src => :E, :tgt => :E), Dict(:src => :V, :tgt => :V))

Graph(Set([:V, :E]), Set([:src, :tgt]), Dict(:src => :E, :tgt => :E), Dict(:src => :V, :tgt => :V))

In [6]:
g = ACSet(Gr)

ACSet(Graph(Set([:V, :E]), Set([:src, :tgt]), Dict(:src => :E, :tgt => :E), Dict(:src => :V, :tgt => :V)), Base.RefValue{Int64}(0), Dict{Symbol, AbstractSet{Int64}}(:V => Set{Int64}(), :E => Set{Int64}()), Dict(:src => Dict(), :tgt => Dict()))

In [27]:
function add_parts!(acs::ACSet, ob::Symbol, n::Int)
    @assert ob ∈ acs.schema.vertices
    i = acs.nextId[]
    acs.nextId[] += n
    union!(acs.parts[ob], i:(i+n-1))
    i:(i+n-1)
end

add_parts! (generic function with 1 method)

In [8]:
struct ACSetTransformation
    schema::Graph
    dom::ACSet
    codom::ACSet
    components::Dict{Symbol, Dict{Int,Int}}
end

In [28]:
g, h = ACSet(Gr), ACSet(Gr)

(ACSet(Graph(Set([:V, :E]), Set([:src, :tgt]), Dict(:src => :E, :tgt => :E), Dict(:src => :V, :tgt => :V)), Base.RefValue{Int64}(0), Dict{Symbol, AbstractSet{Int64}}(:V => Set{Int64}(), :E => Set{Int64}()), Dict(:src => Dict(), :tgt => Dict())), ACSet(Graph(Set([:V, :E]), Set([:src, :tgt]), Dict(:src => :E, :tgt => :E), Dict(:src => :V, :tgt => :V)), Base.RefValue{Int64}(0), Dict{Symbol, AbstractSet{Int64}}(:V => Set{Int64}(), :E => Set{Int64}()), Dict(:src => Dict(), :tgt => Dict())))

In [29]:
add_parts!(g, :V, 3), add_parts!(g, :E, 2)

(0:2, 3:4)

In [ ]:
h = @acset Gr begin
    V = 3
    E = 2
    src = [1,2]
    tgt = [2,3]
end

In [ ]:
g = @acset Gr begin
    V = 2
    E = 1
    src = [1]
    tgt = [2]
end

In [9]:
add_part!(g, :V)

0

In [10]:
g.parts

Dict{Symbol, AbstractSet{Int64}} with 2 entries:
  :V => Set([0])
  :E => Set{Int64}()

In [11]:
add_part!(g, :V)

1

In [12]:
add_part!(g, :E)

2

In [13]:
g.parts

Dict{Symbol, AbstractSet{Int64}} with 2 entries:
  :V => Set([0, 1])
  :E => Set([2])

In [14]:
function set_subpart!(acs::ACSet, i::Int, f::Symbol, j::Int)
    @assert f ∈ acs.schema.edges
    dom = acs.schema.src[f]
    codom = acs.schema.tgt[f]
    @assert i ∈ acs.parts[dom]
    @assert j ∈ acs.parts[codom]
    acs.subparts[f][i] = j
end

set_subpart! (generic function with 1 method)

In [15]:
set_subpart!(g, 2, :src, 0)

0

In [16]:
set_subpart!(g, 2, :tgt, 1)

1

In [17]:
g.subparts

Dict{Symbol, Dict{Int64, Int64}} with 2 entries:
  :src => Dict(2=>0)
  :tgt => Dict(2=>1)